#### HW1 Extract the first page of all pdfs in the "mypdfs " folder and merge them into one pdf file named "new.pdf"

In [3]:
from pathlib import Path
 
path = Path(r"mypdfs")
pdf_files = [p for p in path.glob("*.pdf")]
print(pdf_files)

from PyPDF4 import PdfFileReader, PdfFileMerger, PdfFileWriter
newfilelist = []
for file in pdf_files: 
    with open(file, "rb") as pdf_file: 
        pdf = PdfFileReader(pdf_file) 
        page = pdf.getPage(0) 
    
        writer = PdfFileWriter() 
        writer.addPage(page) 
    

        newfile = file.name[:-4]+'_new.pdf'
        newfilelist.append(newfile)
        with open(newfile, "wb") as output:
             writer.write(output)

merger = PdfFileMerger()
for file_name in newfilelist:
    merger.append(file_name)
merger.write("new.pdf")
merger.close()

import os
for file_name in newfilelist:
    os.remove(file_name)

[WindowsPath('mypdfs/1905.05406.pdf'), WindowsPath('mypdfs/1908.02426.pdf'), WindowsPath('mypdfs/2103.07579.pdf'), WindowsPath('mypdfs/2106.13112.pdf'), WindowsPath('mypdfs/2109.05673.pdf')]


#### HW2 Take the abstracts of the first 20 papers in the given URL, translate them into Chinese, and write them into the file named 'my_abstract.txt’. 
##### URL = https://openaccess.thecvf.com/ICCV2021?day=2021-10-12

### method 1: use translate

In [5]:
import os
import requests
from bs4 import BeautifulSoup
import re

def get_paper_links(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    links = bs.find_all(href=re.compile('/papers/.*.pdf'))
    paper_links = ["https://openaccess.thecvf.com" + link.get('href') for link in links[:20]]
    return paper_links

def main():
    url = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
    paper_links = get_paper_links(url)
    directory = "ICCV"

    if not os.path.exists(directory):
        os.makedirs(directory)

    for i, link in enumerate(paper_links):
        response = requests.get(link)
        with open(os.path.join(directory, f'paper{i+1}.pdf'), 'wb') as f:
            f.write(response.content)

if __name__ == "__main__":
    main()

In [6]:
import fitz  # PyMuPDF
from translate import Translator

def extract_pdf_abstract(pdf_path):
    # Extract text from the first page of the PDF
    doc = fitz.open(pdf_path)
    first_page = doc.load_page(0)
    text = first_page.get_text()
    doc.close()
    return text

def translate_text(text, target_language='zh-cn'):
    # Translate text using the translate library
    translator= Translator(to_lang=target_language)
    translation = translator.translate(text)
    return translation

def main():
    pdf_files = ['./ICCV/' + file for file in os.listdir('./ICCV/')]
    translated_abstracts = []


    for pdf_file in pdf_files:
        abstract = extract_pdf_abstract(pdf_file)
        translated_abstract = translate_text(abstract[:500])
        translated_abstracts.append(translated_abstract)

    # Write translated abstracts to a file
    with open('my_abstract.txt', 'w', encoding='utf-8') as file:
        for i, translated_abstract in enumerate(translated_abstracts):
            file.write(f'Paper {i + 1}:\n')
            file.write(translated_abstract + '\n\n')

if __name__ == "__main__":
    main()


### method 2: use baidu API

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import random
from hashlib import md5
import time

In [22]:
def getUrl(url):
    response = requests.get(url)
    pdfUrl = []
    count = 0
    bs = BeautifulSoup(response.text,"html.parser")  # Parse the HTML content of the response using BeautifulSoup
    pdfs = bs.find_all(href=re.compile('/content/.*html')) # Find all anchor tags with href attribute matching the specified pattern
    names = []
    for tag in pdfs:
        if count < 20:  # Limit the count to 20
            pdfUrl.append("https://openaccess.thecvf.com"+tag.get("href"))
            count = count+1
            # Extract the name from the href attribute and modify it before appending to the names list
            names.append(re.split('[./]', tag.get("href"))[-2][:-16].replace("_"," ")) 
        else: break
    return names,pdfUrl


def getAbstract(paperUrls):
    abstract = []
    for url in paperUrls:
        response = requests.get(url)
        bs = BeautifulSoup(response.text,"html.parser")
        abstract.append(bs.find(id="abstract").text.strip()) #get the content of id "abstract"
    return abstract

def translate_api(input_text):
    # Set your own appid/appkey.
    appid = '20231228001923967'
    appkey = 'cHWnOFT64jrV0FZKsNHN'
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()
    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang = 'zh'
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    query = input_text
    # Generate salt and sign
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    print(result)
    if 'trans_result' in result:
        return result['trans_result'][0]['dst']
    else:
        print("Error: ", result)
        return None


def write2txt(path,title,contents):
    with open(path,"w") as f:
        for ti,con in zip(title,contents):
            f.write(ti+"\n")
            f.write(con+"\n")
            f.write("\n")

In [25]:
web = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
name,pdfurls = getUrl(web)
abstracts = getAbstract(pdfurls)
ch_abstracts = []

In [26]:
for ab in abstracts:
    temp = translate_api(ab)
    ch_abstracts.append(temp)
    time.sleep(1)
write2txt("./my_abstract.txt",name,ch_abstracts)